In [1]:
import pandas as pd
import numpy as np
from __future__ import division
import csv as csv

In [2]:
train = pd.read_csv('train.csv', header=0)

train['Pclass'].fillna(train['Pclass'].median(), inplace=True)

train['Fare'].fillna(train['Fare'].mean(), inplace=True)

train['Gender'] = train['Sex'].map( {'female':0, 'male':1} ).astype(int)
train['Gender'].fillna(train['Gender'].median(), inplace=True)

train['Agefill'] = train['Age']
for i in range(0,2):
    for j in range(0,3):
        median_age = train[(train['Gender'] == i) & (train['Pclass'] == (j+1))]['Age'].dropna().median()
        train.loc[ train.Age.isnull() & (train['Gender'] == i) & (train['Pclass'] == (j+1)) , 'Agefill'] = median_age 

train['FamilySize'] = train['SibSp'] + train['Parch']
train['FamilySize'].fillna(train['FamilySize'].mean(), inplace=True)

train['AgCl'] = train['Agefill'] * train['Pclass']
train['AgCl'].fillna(train['AgCl'].mean(), inplace=True)

train.dtypes[train.dtypes.map(lambda x: x=='object')]

train = train.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis=1)
print train.info()
train = train.values

X = np.transpose(np.array([train[:,2], train[:,5], train[:,6], train[:,7],  train[:,8], train[:,9]]))
#X = np.transpose(np.array([train[:,2], train[:,6]]))
y = train[:,1]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Gender         891 non-null int64
Agefill        891 non-null float64
FamilySize     891 non-null int64
AgCl           891 non-null float64
dtypes: float64(3), int64(7)
memory usage: 69.7 KB
None


In [3]:
X = X - np.mean(X, axis=0)
X = X / (np.max(X, axis=0) - np.min(X, axis=0))

In [4]:
from sklearn.cross_validation import train_test_split

X = np.repeat(X, 50, axis=0)
y = np.repeat(y, 50, axis=0)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

print X_train.shape
print X_val.shape
print y_train.shape
print y_val.shape

(35640, 6)
(8910, 6)
(35640,)
(8910,)


In [ ]:
from sklearn import linear_model

model = linear_model.LogisticRegression()
model.fit(X_train, y_train)

print model.score(X_train, y_train)
print model.score(X_val, y_val)

In [ ]:
from sklearn.metrics import precision_score, recall_score

pred_prob = model.predict_proba(X_val)[:,1]
pred_val = np.array((pred_prob > 0.5), dtype=int)

print precision_score(y_val, pred_val)
print recall_score(y_val, pred_val)

In [ ]:
model.coef_

In [5]:
test = pd.read_csv('test.csv', header=0)

test['Pclass'].fillna(test['Pclass'].median(), inplace=True)

test['Fare'].fillna(test['Fare'].mean(), inplace=True)

test['Gender'] = test['Sex'].map( {'female':0, 'male':1} ).astype(int)
test['Gender'].fillna(test['Gender'].median(), inplace=True)

test['Agefill'] = test['Age']
for i in range(0,2):
    for j in range(0,3):
        median_age = test[(test['Gender'] == i) & (test['Pclass'] == (j+1))]['Age'].dropna().median()
        test.loc[ test.Age.isnull() & (test['Gender'] == i) & (test['Pclass'] == (j+1)) , 'Agefill'] = median_age

test['FamilySize'] = test['SibSp'] + test['Parch']
test['FamilySize'].fillna(test['FamilySize'].mean(), inplace=True)

test['AgCl'] = test['Agefill'] * test['Pclass']
test['AgCl'].fillna(test['AgCl'].mean(), inplace=True)

test.dtypes[test.dtypes.map(lambda x: x=='object')]

test = test.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis=1)
test = test.values

X_test = np.transpose(np.array([test[:,1], test[:,4], test[:,5], test[:,6],  test[:,7], test[:,8]]))

In [ ]:
pred_test_prob = model.predict_proba(X_test)[:,1]
pred_test = np.array((pred_test_prob > 0.5), dtype=int)

In [ ]:
prediction_file = open("schil_log_reg.csv", "wb")
prediction_file_object = csv.writer(prediction_file)
prediction_file_object.writerow(["PassengerId", "Survived"])

test_file = open('test.csv', 'rb')
test_file_object = csv.reader(test_file)
header = test_file_object.next()

i=0

for row in test_file_object:
    prediction_file_object.writerow([row[0], pred_test[i]])
    i = (i+1)
    
prediction_file.close()

# Implement logistic regression with L2 regularization from scratch 

In [6]:
def predict_prob(data, weights):
    scores = np.dot(data, weights)
    sigmoid = 1 / (1+np.exp(-scores))
    return sigmoid

In [7]:
def predict_output(data, weights, th):
    pred_prob = predict_prob(data, weights)
    pred = np.array(pred_prob > th, dtype=int)
    return pred

In [8]:
def feature_derivate_with_L2(feature, error, weight, l2_penalty, is_bias=0):
    derivative = np.dot(feature, error)
    if not is_bias:
        derivative -= 2*l2_penalty*weight
    return derivative

In [9]:
def compute_log_likelihood_with_L2(data, weights, target, l2_penalty):
    #indicator = (target == +1)
    indicator = target
    scores = np.dot(data, weights)
    ll = np.sum((indicator-1)*scores - np.log(1+np.exp(scores))) - l2_penalty*np.dot(weights[1:], np.transpose(weights[1:]))
    return ll

In [10]:
def logistic_regression_with_L2(data, target, initial_weights, step_size, l2_penalty=0, max_itr=10):
    weights = np.array(initial_weights)
    
    #indicator = target == +1
    indicator = target
    
    for i in xrange(max_itr):
        predictions = predict_prob(data, weights)
        error = (indicator - predictions)
        
        for j in xrange(len(weights)):
            partial_derivative = feature_derivate_with_L2(data[:,j], error, weights[j], l2_penalty, (j==0))
            weights[j] += step_size*partial_derivative
        
        ll = compute_log_likelihood_with_L2(data, weights, target, l2_penalty)
    
    print "ll:", np.round(ll,3)
    
    return weights

In [11]:
def logistic_regression_score(data, weights, target, th=0.5):
    pred = predict_output(data, weights, th)
    return (np.sum(pred == target) / len(target))

In [12]:
X_train2 = np.insert(X_train, 0, 1, axis=1)
X_val2 = np.insert(X_val, 0, 1, axis=1)

initial_weights = np.array([0]*X_train2.shape[1], dtype=float)
step_size = 1e-5
max_itr = 5000
l2_penalty_list = [0,1,2,3,4,5,6,7]

for l2_penalty in l2_penalty_list:
    weights = logistic_regression_with_L2(X_train2, y_train, initial_weights, step_size, l2_penalty, max_itr)
    print logistic_regression_score(X_train2, weights, y_train)
    print logistic_regression_score(X_val2, weights, y_val)
    print ""

ll: 8189.159
0.8044332211
0.800224466891

ll: 8115.29
0.8044332211
0.800224466891

ll: 8044.326
0.805555555556
0.801346801347

ll: 7976.012
0.805555555556
0.801346801347

ll: 7910.12
0.805555555556
0.801346801347

ll: 7846.45
0.806593714927
0.802805836139

ll: 7784.825
0.804208754209
0.801122334456

ll: 7725.088
0.805359147026
0.802132435466



In [13]:
X_train2 = np.insert(X_train, 0, 1, axis=1)
X_val2 = np.insert(X_val, 0, 1, axis=1)

initial_weights = np.array([0]*X_train2.shape[1], dtype=float)
step_size = 1e-5
max_itr = 5000
l2_penalty = 5

weights = logistic_regression_with_L2(X_train2, y_train, initial_weights, step_size, l2_penalty, max_itr)
print logistic_regression_score(X_train2, weights, y_train)
print logistic_regression_score(X_val2, weights, y_val)

ll: 7846.45
0.806593714927
0.802805836139


In [14]:
X_test2 = np.insert(X_test, 0, 1, axis=1)
y_test_pred = predict_output(X_test2, weights, 0.5)

In [15]:
prediction_file = open("schil_log_reg_self.csv", "wb")
prediction_file_object = csv.writer(prediction_file)
prediction_file_object.writerow(["PassengerId", "Survived"])

test_file = open('test.csv', 'rb')
test_file_object = csv.reader(test_file)
header = test_file_object.next()

i=0

for row in test_file_object:
    prediction_file_object.writerow([row[0], y_test_pred[i]])
    i = (i+1)
    
prediction_file.close()